# Deploy model

## Settings

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import helpers.settings as sts
sts.print_settings(sts)

BEST_ESTIMATOR_FILENAME : best_estimator_0.0.1.pkl
DATASET_TRAIN_FILENAME : dataset_train.parquet
DATASET_VALIDATION_FILENAME : dataset_validation.parquet
ETL_VERSION : 0.0.1
MODEL_FILENAME : model.pkl
MODEL_VERSION : 0.0.1
PREPROCESSOR_FILENAME : preprocessor_0.0.1.pkl
TRAINED_BEST_ESTIMATOR_FILENAME : trained_best_estimator_0.0.1.pkl
color : <class 'helpers.settings.color'>
print_settings : <function print_settings at 0x7fd0f9aaec10>



## Imports

In [4]:
import pickle
import pandas as pd
import numpy as np
import requests

## Load trained best estimator

In [5]:
with open(f"artifacts/{sts.TRAINED_BEST_ESTIMATOR_FILENAME}","rb") as file:
    best_estimator = pickle.load(file)    

## Dump final model

In [6]:
with open(f"artifacts/{sts.MODEL_FILENAME}","wb") as file:
    pickle.dump(best_estimator, file)

## Load validation dataset

In [7]:
df_validation = pd.read_parquet(f"data/{sts.DATASET_VALIDATION_FILENAME}")

In [8]:
X_validation = df_validation.iloc[:,:-1]
y_validation = df_validation.iloc[:,-1]

## Test deployment

### Batch

In [18]:
complete_batch_json = {
    "instances":X_validation.to_dict(orient="records")
}

In [19]:
response = requests.post("https://coursera-ds-capstone-dev.uc.r.appspot.com/v1/batch_predict", json=complete_batch_json)
response.json(), response.elapsed.total_seconds()

({'prediction': [0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.007360644257703081,
   6.557377049180328e-05,
   0.0,
   0.98,
   0.0,
   0.0,
   0.0,
   0.03710526315789473,
   0.0155,
   0.0,
   0.0205,
   0.0,
   0.0026666666666666666,
   0.0,
   0.0,
   0.0,
   0.0,
   0.005,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0915,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.007333333333333334,
   0.0,
   0.0,
   0.0,
   0.00036734693877551024,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0001648302377696561,
   0.0,
   0.004,
   0.002,
   0.0,
   0.0,
   0.0,
   0.004,
   0.002,
   0.0,
   0.0,
   0.0,
   0.004057971014492754,
   0.0,
   0.0015,
   0.004,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0005,
   0.0,
   0.0,
   0.0,
   0.006,
   1.4962593516209477e-05,
   0.0,
   0.0,
   0.025,
   0.0,
   0.0,
   0.0,
   0.0,
   0.009833333333333333,
  

### Individual prediction

In [30]:
json = {
  "CODE_GENDER": "M",
  "FLAG_OWN_CAR": "Y",
  "FLAG_OWN_REALTY": "Y",
  "CNT_CHILDREN": 0,
  "AMT_INCOME_TOTAL": 0,
  "NAME_INCOME_TYPE": "string",
  "NAME_EDUCATION_TYPE": "string",
  "NAME_FAMILY_STATUS": "string",
  "NAME_HOUSING_TYPE": "string",
  "DAYS_BIRTH": 0,
  "DAYS_EMPLOYED": 0,
  "FLAG_MOBIL": 1,
  "FLAG_WORK_PHONE": 1,
  "FLAG_PHONE": 1,
  "FLAG_EMAIL": 1,
  "OCCUPATION_TYPE": "string",
  "CNT_FAM_MEMBERS": 0
}

In [31]:
response = response = requests.post("https://coursera-ds-capstone-dev.uc.r.appspot.com/v1/predict", json=json)
response.json(), response.elapsed.total_seconds()

({'prediction': 0.035166666666666666,
  'features': {'CODE_GENDER': 'M',
   'FLAG_OWN_CAR': 'Y',
   'FLAG_OWN_REALTY': 'Y',
   'CNT_CHILDREN': 0,
   'AMT_INCOME_TOTAL': 0.0,
   'NAME_INCOME_TYPE': 'string',
   'NAME_EDUCATION_TYPE': 'string',
   'NAME_FAMILY_STATUS': 'string',
   'NAME_HOUSING_TYPE': 'string',
   'DAYS_BIRTH': 0,
   'DAYS_EMPLOYED': 0,
   'FLAG_MOBIL': 1,
   'FLAG_WORK_PHONE': 1,
   'FLAG_PHONE': 1,
   'FLAG_EMAIL': 1,
   'OCCUPATION_TYPE': 'string',
   'CNT_FAM_MEMBERS': 0}},
 0.451982)